In [1]:
# Устанавливаем пакет pymongo
!pip install pymongo

In [2]:
from pymongo import MongoClient

In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from pprint import pprint as pp

**Задание 1**

In [4]:
client = MongoClient('localhost', 27017)

In [5]:
db = client['vacancy_database']

In [6]:
vacancy_coll = db.vacancy
#vacancy_coll.delete_many({})

In [7]:
base_url = 'https://hh.ru'
search = '/search/vacancy'

In [8]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36'}

In [9]:
# Разбивает текст зарплаты на 3 составляющие и убирает из текста '\u202f'
def zp(text):
    text = text.replace('\u202f', '')
    text_list = text.split()
    ot = 0
    do = 0
    if text.find('от') != -1:
        ot = int(text_list[text_list.index('от')+1])
    if text.find('до') != -1:
        do = int(text_list[text_list.index('до')+1])
    if text.find('–') != -1: 
        ot = int(text_list[text_list.index('–')-1])
        do = int(text_list[text_list.index('–')+1])
    val = text_list[-1]
    
    return ot, do, val, text

In [10]:
# Находит последнюю страницу
def get_maxpage():
    params = {'search_field':'name', 'text':inp_vacancy, 'clusters':'true', 'ored_clusters':'true', 'enable_snippets':'true', 'customDomain': '1', 'hhtmFrom': 'vacancy_search_list'}
    response = requests.get(base_url + search, headers = headers, params = params)
    dom = BeautifulSoup(response.text, 'html.parser')
    pages = dom.find_all('a', {'class':"bloko-button"})
    max_page = int(pages[-2].find('span').text)
    
    return max_page

In [11]:
# Проверяет является ли вакансия новой и обновляет данные, если такая ссылка уже есть
# или добавляет новый документ в базу, если такой ссылки ещё нет
def isnew_vac(link_tag, money, vacancy_coll):
    bool_res = False
    if money is None:
        new_data = {
                'link_tag_text': link_tag.text, 
                'link_tag_href': link_tag['href'], 
                'zp_text': None, 
                'zp_ot': '0', 
                'zp_do': '0', 
                'zp_val': None}
    else:
        ot, do, val, text = zp(money.text)
        new_data = {
                'link_tag_text': link_tag.text, 
                'link_tag_href': link_tag['href'], 
                'zp_text': text, 
                'zp_ot': ot, 
                'zp_do': do, 
                'zp_val': val}
    if vacancy_coll.find_one({'link_tag_href': link_tag['href']}):
        vacancy_coll.replace_one({'link_tag_href': link_tag['href']}, new_data) 
        bool_res = True
    else:
        vacancy_coll.insert_one(new_data)
    
    return bool_res

Введите вакансию:

In [12]:
inp_vacancy = 'Системный администратор'

In [13]:
max_page = get_maxpage()
#max_page = 2

In [14]:
res_list = []
for num_page in np.arange(max_page):
    params = {'search_field':'name', 'text':inp_vacancy, 'clusters':'true', 'ored_clusters':'true', 'enable_snippets':'true', 'customDomain': '1', 'page': num_page, 'hhtmFrom': 'vacancy_search_list'}
    response = requests.get(base_url + search, headers = headers, params = params)
    dom = BeautifulSoup(response.text, 'html.parser')
    conteiners = dom.find_all('div', {'class':"vacancy-serp-item"})
    for vac in conteiners:
        link_tag = vac.find('a', {'data-qa':"vacancy-serp__vacancy-title"}, recursive = 'False') 
        money = vac.find('span', {'data-qa': 'vacancy-serp__vacancy-compensation'})
        if isnew_vac(link_tag, money, vacancy_coll):
            res_list = np.append(res_list, link_tag['href'])         

In [15]:
# Список для быстрой проверки сколько найдено вакансий в БД
len(res_list)

0

In [16]:
# Проверяем что записалось в БД и считаем общее количество документов
col = 0
for i in vacancy_coll.find({}):
    col += 1
    pp(i)

{'_id': ObjectId('62180ddb012548666c934b52'),
 'link_tag_href': 'https://hh.ru/vacancy/51331116?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%A1%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%BD%D1%8B%D0%B9%20%D0%B0%D0%B4%D0%BC%D0%B8%D0%BD%D0%B8%D1%81%D1%82%D1%80%D0%B0%D1%82%D0%BE%D1%80',
 'link_tag_text': 'Системный администратор',
 'zp_do': 90000,
 'zp_ot': 70000,
 'zp_text': '70000 – 90000 руб.',
 'zp_val': 'руб.'}
{'_id': ObjectId('62180ddb012548666c934b53'),
 'link_tag_href': 'https://hh.ru/vacancy/53013254?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%A1%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%BD%D1%8B%D0%B9%20%D0%B0%D0%B4%D0%BC%D0%B8%D0%BD%D0%B8%D1%81%D1%82%D1%80%D0%B0%D1%82%D0%BE%D1%80',
 'link_tag_text': 'Помощник системного администратора',
 'zp_do': 50000,
 'zp_ot': 0,
 'zp_text': 'до 50000 руб.',
 'zp_val': 'руб.'}
{'_id': ObjectId('62180ddb012548666c934b54'),
 'link_tag_href': 'https://hh.ru/vacancy/53021820?from=vacancy_search_list&hhtmFrom=vacancy_search_l

 'zp_do': 0,
 'zp_ot': 200000,
 'zp_text': 'от 200000 KZT',
 'zp_val': 'KZT'}
{'_id': ObjectId('62180de7012548666c934bfa'),
 'link_tag_href': 'https://hh.ru/vacancy/52897252?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%A1%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%BD%D1%8B%D0%B9%20%D0%B0%D0%B4%D0%BC%D0%B8%D0%BD%D0%B8%D1%81%D1%82%D1%80%D0%B0%D1%82%D0%BE%D1%80',
 'link_tag_text': 'Системный администратор',
 'zp_do': 50000,
 'zp_ot': 0,
 'zp_text': 'до 50000 руб.',
 'zp_val': 'руб.'}
{'_id': ObjectId('62180de7012548666c934bfb'),
 'link_tag_href': 'https://hh.ru/vacancy/50705020?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%A1%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%BD%D1%8B%D0%B9%20%D0%B0%D0%B4%D0%BC%D0%B8%D0%BD%D0%B8%D1%81%D1%82%D1%80%D0%B0%D1%82%D0%BE%D1%80',
 'link_tag_text': 'Системный администратор',
 'zp_do': 0,
 'zp_ot': 60000,
 'zp_text': 'от 60000 руб.',
 'zp_val': 'руб.'}
{'_id': ObjectId('62180de7012548666c934bfc'),
 'link_tag_href': 'https://hh.ru/vacancy/5

 'link_tag_text': 'Системный администратор',
 'zp_do': 0,
 'zp_ot': 25000,
 'zp_text': 'от 25000 руб.',
 'zp_val': 'руб.'}
{'_id': ObjectId('62180df4012548666c934c98'),
 'link_tag_href': 'https://hh.ru/vacancy/51775176?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%A1%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%BD%D1%8B%D0%B9%20%D0%B0%D0%B4%D0%BC%D0%B8%D0%BD%D0%B8%D1%81%D1%82%D1%80%D0%B0%D1%82%D0%BE%D1%80',
 'link_tag_text': 'Старший системный администратор',
 'zp_do': 120000,
 'zp_ot': 90000,
 'zp_text': '90000 – 120000 руб.',
 'zp_val': 'руб.'}
{'_id': ObjectId('62180df4012548666c934c99'),
 'link_tag_href': 'https://hh.ru/vacancy/52595154?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%A1%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%BD%D1%8B%D0%B9%20%D0%B0%D0%B4%D0%BC%D0%B8%D0%BD%D0%B8%D1%81%D1%82%D1%80%D0%B0%D1%82%D0%BE%D1%80',
 'link_tag_text': 'Младший системный администратор',
 'zp_do': 30000,
 'zp_ot': 30000,
 'zp_text': '30000 – 30000 руб.',
 'zp_val': 'руб.'}
{'_id':

{'_id': ObjectId('62180e04012548666c934d5f'),
 'link_tag_href': 'https://hh.ru/vacancy/52674717?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%A1%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%BD%D1%8B%D0%B9%20%D0%B0%D0%B4%D0%BC%D0%B8%D0%BD%D0%B8%D1%81%D1%82%D1%80%D0%B0%D1%82%D0%BE%D1%80',
 'link_tag_text': 'Системный администратор',
 'zp_do': 35000,
 'zp_ot': 25000,
 'zp_text': '25000 – 35000 руб.',
 'zp_val': 'руб.'}
{'_id': ObjectId('62180e04012548666c934d60'),
 'link_tag_href': 'https://hh.ru/vacancy/50626697?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%A1%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%BD%D1%8B%D0%B9%20%D0%B0%D0%B4%D0%BC%D0%B8%D0%BD%D0%B8%D1%81%D1%82%D1%80%D0%B0%D1%82%D0%BE%D1%80',
 'link_tag_text': 'Системный администратор',
 'zp_do': 0,
 'zp_ot': 40000,
 'zp_text': 'от 40000 руб.',
 'zp_val': 'руб.'}
{'_id': ObjectId('62180e04012548666c934d61'),
 'link_tag_href': 'https://hh.ru/vacancy/49682004?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%

 'zp_ot': '0',
 'zp_text': None,
 'zp_val': None}
{'_id': ObjectId('62180e13012548666c934e27'),
 'link_tag_href': 'https://hh.ru/vacancy/52044152?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%A1%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%BD%D1%8B%D0%B9%20%D0%B0%D0%B4%D0%BC%D0%B8%D0%BD%D0%B8%D1%81%D1%82%D1%80%D0%B0%D1%82%D0%BE%D1%80',
 'link_tag_text': 'Системный администратор',
 'zp_do': '0',
 'zp_ot': '0',
 'zp_text': None,
 'zp_val': None}
{'_id': ObjectId('62180e13012548666c934e28'),
 'link_tag_href': 'https://hh.ru/vacancy/51896467?from=vacancy_search_list&hhtmFrom=vacancy_search_list&query=%D0%A1%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%BD%D1%8B%D0%B9%20%D0%B0%D0%B4%D0%BC%D0%B8%D0%BD%D0%B8%D1%81%D1%82%D1%80%D0%B0%D1%82%D0%BE%D1%80',
 'link_tag_text': 'Дежурный системный администратор Linux',
 'zp_do': '0',
 'zp_ot': '0',
 'zp_text': None,
 'zp_val': None}
{'_id': ObjectId('62180e13012548666c934e29'),
 'link_tag_href': 'https://hh.ru/vacancy/50839841?from=vacancy_search_list&hhtmFr

In [17]:
# Всего документов
col

800

**Задание 2**

In [107]:
# Функция отбирает вакансии, больше заданного значения и выводит их в датафрейм с сортировкой по убыванию
def gl_zp(param):
    list_res = []
    for i in vacancy_coll.find( {'$or' : [{ '$and': [{'zp_ot': { '$gte': param}}, {'zp_do': {'$eq': 0}}]} , {'zp_do': { '$gt': param}}]} ):
        list_res.append([i['link_tag_text'], i['zp_text'], i['zp_ot'], i['zp_do']])
    res_df = pd.DataFrame(list_res, columns = ['Вакансия', 'Зарплата', 'От', 'До'])
    res_df = res_df.sort_values(['От', 'До'], ascending=False)[['Вакансия', 'Зарплата']]
    res_df = res_df.reset_index(drop=True)
    
    return res_df

In [108]:
gl_zp(150000)

,Вакансия,Зарплата
0,SRE / Системный администратор Linux,300000 – 550000 руб.
1,Системный администратор,от 300000 KZT
2,Системный администратор,от 280000 KZT
3,Системный (старший) администратор,200000 – 300000 KZT
4,Системный администратор,200000 – 200000 KZT
5,Системный администратор,от 200000 KZT
6,Системный администратор облачной инфраструктур...,от 200000 руб.
7,DevOps / SRE / Системный администратор,от 200000 руб.
8,Системный администратор,от 200000 KZT
9,Системный администратор Linux/DevOps,от 160000 руб.
